# Characterizing the Hidden Structure of Emerging Epilepsy with Transformers

Outline:
- Embed 1s snippets with Contrastive Learning
- Train Transformer to predict next embedding based on past
- Group embeddings into few categories
- Analyse Attention matrix to understand interactions among pattens